In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import *
import time

%matplotlib inline

In [2]:
jailbooking = pd.read_csv("booking.csv")
entries = pd.read_csv("entries_201805311548.csv")
mentalhealth = pd.read_csv('health.csv')
inmate = pd.read_csv('jocojims2inmatedata_201805311745.csv')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (9,10,14,18,19,26,30,31,32,34,35,36,37,38,40,49,52,64,65,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
inmate.columns

Index(['inmate_no', 'dob', 'age', 'original_sex', 'sex', 'original_race',
       'race', 'hair', 'eyes', 'height', 'weight', 'mar_stat', 'city', 'state',
       'zip', 'country', 'birth_place', 'arr_agency', 'original_case_number',
       'case_no', 'arrest_dt', 'arrest_time', 'trans_agency', 'control_',
       'bk_dt', 'book_time', 'bonding', 'arr_location', 'b_state', 'mni_no',
       'location', 'license_state', 'veh_lic__', 'veh_lic_state',
       'veh_lic_year', 'veh_make', 'veh_model', 'veh_color', 'dna_draw',
       'l_e_case_', 'same_as_', 'rel_date', 'release_time',
       'associated_records', 'driver_s_lic_type',
       'new_booked_inmate___orientation', 'exp_stamp', 'complexion', 'cfn',
       'demographic_code', 'pros_wksht__', 'ethnic', 'mnh_refer_dt', 'lgbt',
       'at_risk', 'exploited', 'past_year', 'concerns', 'int_dt',
       'notify_asap', 'notify_verbal', 'religion', 'diet', 'rstr_liv_cond',
       'printed_chow', 'prev_bkno', 'ofndr_reg_req', 'ofndr_stamp',
     

In [4]:
inmate = inmate[['race', 'mar_stat', 'city', 'state', 'zip', 'dedupe_id']]

In [5]:
inmate = inmate.drop_duplicates(subset=['dedupe_id'])

In [6]:
inmate.shape

(144254, 6)

In [7]:
inmate.dedupe_id.nunique()

144253

In [8]:
entries.rename(columns={'foreign_id' : 'mni_no'}, inplace=True)
entries = entries[["mni_no", "dedupe_id", 'dob',  'sex']]

In [9]:
entries.shape

(899470, 4)

In [10]:
ent_inmate = pd.merge(entries, inmate, left_on='dedupe_id', right_on='dedupe_id')

In [11]:
ent_inmate.shape

(366077, 9)

In [12]:
ent_inmate.to_csv("ent_inmate.csv")

In [13]:
jailbooking.columns

Index(['mni_no', 'case_no', 'case_type', 'booking_no', 'arresting_agency',
       'arresting_agency_type', 'booking_date', 'release_date', 'bail_type',
       'bail_amt', 'bailed_out'],
      dtype='object')

In [14]:
jailbooking = jailbooking.dropna(subset=['mni_no'])

In [15]:
jailbooking.shape

(28579, 11)

In [16]:
ent_inmate[ent_inmate['dedupe_id']==487227]

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip
184403,669258,487227.0,1973-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103
184404,669263,487227.0,1974-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103
184405,141326,487227.0,1974-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103
184406,669265,487227.0,1974-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103
184407,669261,487227.0,1974-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103
184408,669260,487227.0,1974-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103
184409,669262,487227.0,1974-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103
184410,669264,487227.0,1974-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103
184411,669259,487227.0,1974-02-09,MALE,WHITE,S,KANSAS CITY,KS,66103


In [17]:
withbooking = pd.merge(ent_inmate, jailbooking, on='mni_no')

In [18]:
withbooking[withbooking['dedupe_id']==309]

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,case_type,booking_no,arresting_agency,arresting_agency_type,booking_date,release_date,bail_type,bail_amt,bailed_out
11129,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061,14DV00270,DV,14002590,OLATHE P.D.,CITY,2014-03-04 00:00:00,2014-03-05 00:00:00,PR,1500.0,True
11130,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061,15DV00382,DV,15003546,LENEXA P.D.,CITY,2015-03-26 00:00:00,2015-03-27 00:00:00,SUR,1000.0,True
11131,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061,16DV00234,DV,16002371,LENEXA P.D.,CITY,2016-02-24 00:00:00,2016-04-28 00:00:00,NaN,NaN,False


In [19]:
mentalhealth.columns

Index(['hash_ssn', 'hash_ssn4', 'hash_lname', 'hash_fname', 'patid', 'dob',
       'race', 'sex', 'hash_gcstreetaddress', 'city', 'state', 'zip',
       'gcvalidity', 'gcsrctabl', 'gcscore', 'gccity', 'gccounty', 'gcstate',
       'gczipcode', 'hash_gclonglat', 'gctract2010id', 'gcblockgroup2010id',
       'gcblock2010id', 'admit_date', 'dschrg_date', 'program', 'pri_dx_code',
       'pri_dx_value', 'refferal_source', 'case_id', 'dedupe_id'],
      dtype='object')

In [20]:
mentalhealth = mentalhealth[['dedupe_id','pri_dx_value', 'refferal_source']]

In [21]:
mentalhealth.head()

,dedupe_id,pri_dx_value,refferal_source
0,190642.0,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN
1,1780.0,DEPRESSIVE DISORDER NOS,NaN
2,583264.0,"BIPOLAR DISORDER, DEPRESSED, SEVERE W/O PSYCHO...",NaN
3,666871.0,"SCHIZOPHRENIC DISORDER, SCHIZO-AFFECTIVE TYPE,...",NaN
4,114636.0,"MANIC DISORDER, SINGLE EPISODE, IN FULL REMISSION",SELF


In [22]:
mentalhealth['refferal_source'].loc[mentalhealth['refferal_source'] != 'SELF'] = 'OTHERS'

In [23]:
mentalhealth.head()

,dedupe_id,pri_dx_value,refferal_source
0,190642.0,UNSPECIFIED AFFECTIVE PSYCHOSIS,OTHERS
1,1780.0,DEPRESSIVE DISORDER NOS,OTHERS
2,583264.0,"BIPOLAR DISORDER, DEPRESSED, SEVERE W/O PSYCHO...",OTHERS
3,666871.0,"SCHIZOPHRENIC DISORDER, SCHIZO-AFFECTIVE TYPE,...",OTHERS
4,114636.0,"MANIC DISORDER, SINGLE EPISODE, IN FULL REMISSION",SELF


In [24]:
mentalhealth['pri_dx_value'].fillna('missing', inplace = True)

In [25]:
mentalhealth['refferal_source'].fillna('OTHERS', inplace = True)

In [26]:
mentalhealth.drop_duplicates(subset=['dedupe_id'])

,dedupe_id,pri_dx_value,refferal_source
0,190642.0,UNSPECIFIED AFFECTIVE PSYCHOSIS,OTHERS
1,1780.0,DEPRESSIVE DISORDER NOS,OTHERS
2,583264.0,"BIPOLAR DISORDER, DEPRESSED, SEVERE W/O PSYCHO...",OTHERS
3,666871.0,"SCHIZOPHRENIC DISORDER, SCHIZO-AFFECTIVE TYPE,...",OTHERS
4,114636.0,"MANIC DISORDER, SINGLE EPISODE, IN FULL REMISSION",SELF
5,38306.0,PANIC DISORDER,OTHERS
6,197461.0,"MAJOR DEPRESSIVE DISORDER, RECURRENT, MODERATE",OTHERS
7,74789.0,ATTENTION DEFICIT DISORDER WITH HYPERACTIVITY,OTHERS
8,439589.0,PROLONGED POST TRAUMATIC STRESS DISORDER,OTHERS
9,716225.0,"SCHIZOPHRENIC DISORDER, PARANOID TYPE",OTHERS


In [27]:
almost_done = pd.merge(withbooking, mentalhealth, on='dedupe_id', how='left')

In [28]:
almost_done.shape

(34823, 21)

In [29]:
withbooking=withbooking.drop_duplicates()

In [30]:
withbooking[withbooking['mni_no']==658145]

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,case_type,booking_no,arresting_agency,arresting_agency_type,booking_date,release_date,bail_type,bail_amt,bailed_out
3,658145,350849.0,1997-04-12,FEMALE,WHITE,S,EDGERTON,KS,66021,15CR02335,CR,15011570,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2015-10-07 00:00:00,2015-10-07 00:00:00,SUR,10000.0,True


In [31]:
withbooking.dedupe_id.nunique()

21258

In [32]:
withbooking.shape

(27904, 19)

In [33]:
withbooking.mni_no.nunique()

21130

In [34]:
almost_done

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,booking_no,arresting_agency,arresting_agency_type,booking_date,release_date,bail_type,bail_amt,bailed_out,pri_dx_value,refferal_source
0,679537,301831.0,1974-04-02,MALE,WHITE,M,SHAWNEE,KS,66226,15CR00773,...,15003768,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2015-04-01 00:00:00,2015-04-01 00:00:00,CA,10000.0,True,"MAJOR DEPRESSIVE DISORDER, SINGLE EPISODE, MOD...",OTHERS
1,431835,296410.0,1971-01-01,MALE,WHITE,NaN,OLATHE,KS,66062,13CR00129,...,13000836,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2013-01-19 00:00:00,2013-01-22 00:00:00,SUR,10000.0,True,SOCIAL PHOBIA,OTHERS
2,431835,296410.0,1971-01-01,MALE,WHITE,NaN,OLATHE,KS,66062,14CR00932,...,14005131,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2014-05-02 00:00:00,2014-05-02 00:00:00,SUR,5000.0,True,SOCIAL PHOBIA,OTHERS
3,658145,350849.0,1997-04-12,FEMALE,WHITE,S,EDGERTON,KS,66021,15CR02335,...,15011570,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2015-10-07 00:00:00,2015-10-07 00:00:00,SUR,10000.0,True,NaN,NaN
4,658145,350849.0,1997-04-12,FEMALE,WHITE,S,EDGERTON,KS,66021,15CR02335,...,15011570,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2015-10-07 00:00:00,2015-10-07 00:00:00,SUR,10000.0,True,NaN,NaN
5,655865,370683.0,1984-08-03,MALE,WHITE,S,OLATHE,KS,66062,13CR00468,...,13002475,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2013-03-03 00:00:00,2013-03-03 00:00:00,SUR,1000.0,True,NaN,NaN
6,631990,467807.0,1975-02-18,MALE,WHITE,M,LEAWOOD,KS,66209,11DV01549,...,11014542,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2011-12-01 00:00:00,2011-12-01 00:00:00,SUR,3500.0,True,NaN,NaN
7,510773,103594.0,1991-06-11,MALE,WHITE,S,OVERLAND PARK,KS,66212,15DV00580,...,15005330,OVERLAND PARK P.D.,CITY,2015-05-07 00:00:00,2015-05-08 00:00:00,PR,2500.0,True,NaN,NaN
8,694772,112221.0,1952-02-01,MALE,WHITE,M,OVERLAND PARK,KS,66210,15CR01391,...,15006763,PRAIRIE VILLAGE P.D.,CITY,2015-06-11 00:00:00,2015-06-12 00:00:00,SUR,5000.0,True,NaN,NaN
9,694772,112221.0,1952-02-01,MALE,WHITE,M,OVERLAND PARK,KS,66210,15CR02428,...,15011602,OVERLAND PARK P.D.,CITY,2015-10-08 00:00:00,2015-10-08 00:00:00,SUR,25000.0,True,NaN,NaN


In [35]:
almost_done['mh_treatment'] = pd.notna(almost_done["refferal_source"])

In [36]:
almost_done.shape

(34823, 22)

In [37]:
almost_done = almost_done.drop_duplicates()

In [38]:
almost_done = almost_done.dropna(subset = ['dedupe_id'])

In [39]:
almost_done.shape

(29504, 22)

In [40]:
almost_done.columns

Index(['mni_no', 'dedupe_id', 'dob', 'sex', 'race', 'mar_stat', 'city',
       'state', 'zip', 'case_no', 'case_type', 'booking_no',
       'arresting_agency', 'arresting_agency_type', 'booking_date',
       'release_date', 'bail_type', 'bail_amt', 'bailed_out', 'pri_dx_value',
       'refferal_source', 'mh_treatment'],
      dtype='object')

In [41]:
def convert_datetime(df, cols):
    for col in cols:
        df[col] = pd.to_datetime(df[col])

In [42]:
convert_datetime(almost_done, ['booking_date', 'release_date', 'dob'])

In [43]:
#3 Pre-Process & Filling Missing Data#3 Pre-P 
def Detect_missing_value(df):
    '''
    Find out the columns have missing values

    Input:
        df: (pandas dataframe)
 
    Returns:
        a list of those column names
    '''
    rv = []
    for col in df.columns:
        if df[col].count() < df.shape[0]:
            rv.append(col)
            print(col, "has missing values.")
    return rv

In [44]:
Detect_missing_value(almost_done)

sex has missing values.
mar_stat has missing values.
city has missing values.
state has missing values.
zip has missing values.
arresting_agency has missing values.
arresting_agency_type has missing values.
release_date has missing values.
bail_type has missing values.
bail_amt has missing values.
pri_dx_value has missing values.
refferal_source has missing values.


['sex',
 'mar_stat',
 'city',
 'state',
 'zip',
 'arresting_agency',
 'arresting_agency_type',
 'release_date',
 'bail_type',
 'bail_amt',
 'pri_dx_value',
 'refferal_source']

In [45]:
# get numeric columns 
num_cols = almost_done._get_numeric_data().columns
num_cols

Index(['mni_no', 'dedupe_id', 'booking_no', 'bail_amt', 'bailed_out',
       'mh_treatment'],
      dtype='object')

In [46]:
#and then find out categorical columns
cate_cols = list(set(almost_done.columns) - set(num_cols))
cate_cols

['mar_stat',
 'race',
 'case_type',
 'bail_type',
 'case_no',
 'city',
 'dob',
 'arresting_agency_type',
 'arresting_agency',
 'refferal_source',
 'sex',
 'pri_dx_value',
 'release_date',
 'state',
 'booking_date',
 'zip']

In [47]:
# FILL IN MISSING VALUES WITH SELECTED METHODS FOR EACH TYPE 
def Fill_in(df, cols, method="mean"):
    '''
    Filling in missing values with "mean" or "median"
    
    Inputs:
        df: (pandas dataframe)
        a list of those column names
        method (string): mean or median
 
    Returns:
        pandas dataframe
    '''
    for col in cols:
        if df[col].dtype == 'object':
            try:
                df[col].fillna(df[col].mode()[0], inplace=True)
            except:
                df[col].fillna("")
        if method =='mean':
            val = df[col].mean()
        elif method =='median':
            val = df[col].median()
        elif method =='mode':
            val = df[col].mode()
            val = str(val)
        elif method == 'missing':
            val = "missing"
        df[col] = df[col].fillna(val)
        print ('Filling missing value for {} using {}'.format(col, method))
    return df

In [48]:
almost_done = Fill_in(almost_done, ['city', 'state', 'race', 'mar_stat'], "missing")

Filling missing value for city using missing
Filling missing value for state using missing
Filling missing value for race using missing
Filling missing value for mar_stat using missing


In [49]:
fill_with_mode = ['sex', 'arresting_agency', 'arresting_agency_type', 'bail_type', 'pri_dx_value'] 

In [50]:
almost_done.sex.unique()

array(['MALE', 'FEMALE', nan], dtype=object)

In [51]:
almost_done = Fill_in(almost_done, fill_with_mode, "mode")
almost_done = Fill_in(almost_done, ['bail_amt'], "median")

Filling missing value for sex using mode
Filling missing value for arresting_agency using mode
Filling missing value for arresting_agency_type using mode
Filling missing value for bail_type using mode
Filling missing value for pri_dx_value using mode
Filling missing value for bail_amt using median


In [52]:
Detect_missing_value(almost_done)

zip has missing values.
release_date has missing values.
refferal_source has missing values.


['zip', 'release_date', 'refferal_source']

In [53]:
almost_done['bailed_out'].head()

0    True
1    True
2    True
3    True
5    True
Name: bailed_out, dtype: bool

In [54]:
#convert dummy var
almost_done['bailed_out'] = almost_done['bailed_out']*1

In [55]:
#convert dummy var
almost_done['mh_treatment'] = almost_done['mh_treatment']*1

In [56]:
almost_done[almost_done['dedupe_id']==296410]

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,arresting_agency,arresting_agency_type,booking_date,release_date,bail_type,bail_amt,bailed_out,pri_dx_value,refferal_source,mh_treatment
1,431835,296410.0,1971-01-01,MALE,WHITE,S,OLATHE,KS,66062,13CR00129,...,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2013-01-19,2013-01-22,SUR,10000.0,1,SOCIAL PHOBIA,OTHERS,1
2,431835,296410.0,1971-01-01,MALE,WHITE,S,OLATHE,KS,66062,14CR00932,...,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2014-05-02,2014-05-02,SUR,5000.0,1,SOCIAL PHOBIA,OTHERS,1


In [57]:
almost_done['re_entry'] = almost_done['dedupe_id'].map(almost_done['dedupe_id'].value_counts())

In [58]:
almost_done['re_entry'] = np.where(almost_done['re_entry']>1, 1, 0)

In [59]:
#create age at the released_date
almost_done['age']= almost_done['release_date'].dt.year - almost_done['dob'].dt.year

In [60]:
almost_done.head()

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,booking_date,release_date,bail_type,bail_amt,bailed_out,pri_dx_value,refferal_source,mh_treatment,re_entry,age
0,679537,301831.0,1974-04-02,MALE,WHITE,M,SHAWNEE,KS,66226,15CR00773,...,2015-04-01,2015-04-01,CA,10000.0,1,"MAJOR DEPRESSIVE DISORDER, SINGLE EPISODE, MOD...",OTHERS,1,0,41.0
1,431835,296410.0,1971-01-01,MALE,WHITE,S,OLATHE,KS,66062,13CR00129,...,2013-01-19,2013-01-22,SUR,10000.0,1,SOCIAL PHOBIA,OTHERS,1,1,42.0
2,431835,296410.0,1971-01-01,MALE,WHITE,S,OLATHE,KS,66062,14CR00932,...,2014-05-02,2014-05-02,SUR,5000.0,1,SOCIAL PHOBIA,OTHERS,1,1,43.0
3,658145,350849.0,1997-04-12,FEMALE,WHITE,S,EDGERTON,KS,66021,15CR02335,...,2015-10-07,2015-10-07,SUR,10000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,0,18.0
5,655865,370683.0,1984-08-03,MALE,WHITE,S,OLATHE,KS,66062,13CR00468,...,2013-03-03,2013-03-03,SUR,1000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,0,29.0


In [61]:
almost_done.to_csv('clean_total.csv', index=False)

In [62]:
def within_frame(df, id_num='dedupe_id', timestamp='booking_date', col_name='re-entry', duration=365):
    df = df.sort_values(by=[id_num, timestamp])
    df['{}'.format(col_name)] = df.groupby(id_num)[timestamp].diff()
    df['{}'.format(col_name)] = df['{}'.format(col_name)].apply(lambda x: x.days)
    df['within_{}'.format(duration)] = np.where(df['{}'.format(col_name)]>duration, 1, 0)    

    return df

In [63]:
##################up to now